In [1]:
from brian_graph import *
from solver import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
solve_from_file("inputs/2_50.in", "outputs", ["ILP"])

Processing inputs/2_50.in
Academic license - for non-commercial use only
Optimize a model with 129 rows, 3850 columns and 7602 nonzeros
Model has 63700 quadratic objective terms
Variable types: 0 continuous, 3850 integer (3850 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+02, 1e+04]
  QObjective range [5e+02, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 298461.61402
Presolve removed 29 rows and 1350 columns
Presolve time: 0.08s
Presolved: 1229 rows, 3629 columns, 66058 nonzeros
Variable types: 1108 continuous, 2521 integer (2501 binary)

Root relaxation: objective 5.363042e+03, 403 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 5363.04246    0   53 298461.614 5363.04246  98.2%     -    0s
H    0     0                    131105.13275 5363.